In [2]:
import random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from copy import deepcopy

In [3]:
def generate_random_graph(num_nodes, probability):
    graph = [[0] * num_nodes for _ in range(num_nodes)]
    
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            if random.random() < probability:
                graph[i][j] = 1
                graph[j][i] = 1

    return graph

def generate_random_solution(num_nodes):
    nodes = list(range(num_nodes))
    random.shuffle(nodes)
    return nodes

def calculate_total_edge_length(graph, solution):
    total_length = 0
    for i in range(len(graph)):
        for j in range(i + 1, len(graph)):
            if graph[i][j] == 1:
                position_i = solution.index(i)
                position_j = solution.index(j)
                total_length += abs(position_i - position_j)
    return total_length

# Example usage:
num_nodes = 10
edge_probability = 0.5

# Generate a random graph
graph = generate_random_graph(num_nodes, edge_probability)

# Generate a random solution
random_solution = generate_random_solution(num_nodes)

# Calculate the total edge length for the random solution
total_edge_length = calculate_total_edge_length(graph, random_solution)

print("Random Graph (Adjacency Matrix):")
for row in graph:
    print(row)

print("\nRandom Solution (Permutation):", random_solution)
print("Total Edge Length:", total_edge_length)

Random Graph (Adjacency Matrix):
[0, 1, 0, 0, 0, 1, 1, 1, 1, 0]
[1, 0, 0, 1, 1, 1, 0, 0, 0, 1]
[0, 0, 0, 0, 1, 1, 0, 0, 0, 1]
[0, 1, 0, 0, 1, 1, 1, 1, 0, 1]
[0, 1, 1, 1, 0, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 0, 1, 0, 0, 0]
[1, 0, 0, 1, 1, 1, 0, 0, 1, 0]
[1, 0, 0, 1, 1, 0, 0, 0, 1, 1]
[1, 0, 0, 0, 1, 0, 1, 1, 0, 0]
[0, 1, 1, 1, 1, 0, 0, 1, 0, 0]

Random Solution (Permutation): [5, 6, 3, 1, 4, 8, 9, 0, 2, 7]
Total Edge Length: 86


In [4]:
import itertools

def brute_force(graph):
    num_nodes = len(graph)
    best_solution = None
    best_length = float('inf')

    # Generate all permutations of nodes
    all_permutations = itertools.permutations(range(num_nodes))

    # Check total edge length for each permutation
    for solution in all_permutations:
        total_length = calculate_total_edge_length(graph, solution)
        if total_length < best_length:
            best_length = total_length
            best_solution = solution

    return best_solution, best_length

In [5]:
brute_force(graph)

((2, 9, 1, 3, 4, 5, 6, 0, 7, 8), 68)

In [6]:
def make_change_swap(graph, solution):
    new_solution = solution.copy()
    
    # Choose two distinct random indices
    index1, index2 = random.sample(range(len(solution)), 2)
    
    # Swap the positions of the selected nodes
    new_solution[index1], new_solution[index2] = new_solution[index2], new_solution[index1]

    return new_solution

In [7]:
def make_change_inverse(graph, solution):
    new_solution = solution.copy()
    
    # Choose two distinct random indices
    index1, index2 = sorted(random.sample(range(len(solution)), 2))
    
    # Reverse the subset of nodes between index1 and index2
    new_solution[index1:index2+1] = reversed(new_solution[index1:index2+1])

    return new_solution

In [25]:
def make_change_next_permutation(graph, solution):
    #based on the classic next permutation algorithm
    new_solution = deepcopy(solution)
    
    n = len(new_solution)
    
    # Find the largest index k such that a[k] < a[k+1]
    k = n - 2
    while k >= 0 and new_solution[k] >= new_solution[k + 1]:
        k -= 1

    # If no such index exists, the permutation is the last one
    if k == -1:
        return sorted(new_solution)

    # Find the largest index l greater than k such that a[k] < a[l]
    l = n - 1
    while new_solution[k] >= new_solution[l]:
        l -= 1

    # Swap a[k] and a[l]
    new_solution[k], new_solution[l] = new_solution[l], new_solution[k]

    # Reverse the sequence from a[k+1] up to and including the final element a[n-1]
    new_solution[k + 1:] = reversed(new_solution[k + 1:])

    return new_solution


In [9]:
def local_search(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(num_iters):
        print(solution, value, i)
        new_solution = change_func(graph, solution)
        new_value = calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

    return best_solution, best_value, best_i

In [10]:
def local_search_first_improvement(graph, random_solution, value, num_iters, change_func):
    pass

In [11]:
def local_search_best_improvement(graph, random_solution, value, num_iters, change_func):
    pass

In [12]:
local_search(graph, random_solution, total_edge_length, 1000, change_func=make_change_swap)

[5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 0
[5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 1
[5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 2
[5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 3
[5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 4
[5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 5
[5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 6
[5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 7
[5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 8
[5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 9
[5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 10
[5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 11
[5, 6, 3, 1, 4, 7, 9, 0, 2, 8] 84 12
[5, 6, 3, 1, 4, 7, 9, 0, 2, 8] 84 13
[5, 6, 3, 1, 4, 7, 9, 0, 2, 8] 84 14
[5, 6, 3, 1, 4, 9, 7, 0, 2, 8] 82 15
[5, 6, 3, 1, 4, 9, 7, 0, 2, 8] 82 16
[5, 6, 3, 1, 4, 9, 7, 0, 2, 8] 82 17
[5, 6, 3, 1, 4, 9, 7, 0, 2, 8] 82 18
[6, 5, 3, 1, 4, 9, 7, 0, 2, 8] 81 19
[6, 5, 3, 1, 4, 9, 7, 0, 8, 2] 80 20
[6, 5, 3, 1, 4, 9, 7, 0, 8, 2] 80 21
[6, 5, 3, 1, 4, 9, 7, 0, 8, 2] 80 22
[6, 5, 3, 1, 4, 9, 7, 0, 8, 2] 80 23
[6, 5, 3, 1, 4, 9, 7, 0, 8, 2] 80 24
[6, 5, 3, 1, 4, 9, 7, 0, 8, 2] 80 25
[6, 5, 3, 1, 4, 9, 7, 0, 8, 2] 80 26
[6, 5, 3, 1

([2, 9, 1, 3, 4, 5, 7, 0, 6, 8], 68, 126)

In [13]:
local_search(graph, random_solution, total_edge_length, 1000, change_func=make_change_inverse)

[5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 0
[5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 1
[5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 2
[5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 3
[5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 4
[5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 5
[5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 6
[5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 7
[5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 8
[5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 9
[5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 10
[5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 11
[5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 12
[5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 13
[5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 14
[5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 15
[5, 6, 3, 1, 4, 2, 0, 9, 8, 7] 83 16
[5, 6, 3, 1, 4, 2, 0, 9, 7, 8] 82 17
[5, 6, 3, 1, 4, 2, 0, 9, 7, 8] 82 18
[5, 6, 3, 1, 4, 2, 0, 9, 7, 8] 82 19
[5, 6, 3, 1, 4, 2, 0, 9, 7, 8] 82 20
[5, 6, 3, 1, 4, 2, 0, 9, 7, 8] 82 21
[5, 6, 3, 1, 4, 2, 0, 9, 7, 8] 82 22
[5, 6, 3, 1, 4, 2, 0, 9, 7, 8] 82 23
[5, 6, 3, 1, 4, 2, 0, 9, 7, 8] 82 24
[5, 6, 3, 1, 4, 2, 0, 9, 7, 8] 82 25
[5, 6, 3, 1, 4, 2, 0, 9, 7, 8] 82 26
[5, 6, 3, 1

([2, 9, 1, 4, 3, 5, 6, 0, 7, 8], 68, 90)

In [63]:
def local_search_with_permutation(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(num_iters):
        print(solution, value, i)
        new_solution = change_func(graph, solution)
        new_value = calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

        else:
            perm_counter = 0
            perm_limit = num_iters / (i+1)
            while new_value >= value and perm_counter < perm_limit:    
                new_solution = make_change_next_permutation(graph, new_solution)
                new_value = calculate_total_edge_length(graph, new_solution)
                perm_counter += 1

            if new_value < value:
                value = new_value
                solution = deepcopy(new_solution)

                if new_value < best_value:
                    best_i = i
                    best_value = new_value
                    best_solution = deepcopy(new_solution)
              

    return best_solution, best_value, best_i



In [64]:
local_search_with_permutation(graph, random_solution, total_edge_length, 1000, change_func=make_change_inverse)

[5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 0
[5, 6, 3, 1, 4, 8, 0, 9, 2, 7] 84 1
[5, 6, 3, 9, 1, 4, 2, 0, 7, 8] 83 2
[1, 9, 3, 6, 5, 4, 2, 0, 7, 8] 79 3
[1, 9, 3, 6, 5, 4, 2, 0, 7, 8] 79 4
[1, 9, 3, 6, 5, 4, 2, 0, 7, 8] 79 5
[1, 9, 3, 6, 5, 4, 2, 0, 7, 8] 79 6
[1, 9, 3, 6, 5, 4, 2, 0, 7, 8] 79 7
[1, 9, 3, 6, 5, 4, 2, 0, 7, 8] 79 8
[1, 9, 3, 6, 5, 4, 2, 0, 7, 8] 79 9
[1, 9, 3, 4, 5, 6, 2, 0, 7, 8] 77 10
[1, 9, 3, 4, 5, 2, 6, 0, 7, 8] 75 11
[1, 9, 3, 4, 5, 2, 6, 0, 7, 8] 75 12
[1, 9, 3, 4, 5, 2, 6, 0, 7, 8] 75 13
[1, 9, 3, 4, 5, 2, 6, 0, 7, 8] 75 14
[1, 9, 3, 4, 5, 2, 6, 0, 7, 8] 75 15
[1, 9, 3, 4, 5, 2, 6, 0, 7, 8] 75 16
[1, 9, 3, 4, 5, 2, 6, 0, 7, 8] 75 17
[1, 9, 3, 4, 5, 2, 6, 0, 7, 8] 75 18
[1, 9, 3, 4, 5, 2, 6, 0, 7, 8] 75 19
[1, 9, 3, 4, 5, 2, 6, 0, 7, 8] 75 20
[1, 9, 3, 4, 5, 2, 6, 0, 7, 8] 75 21
[1, 9, 3, 4, 5, 2, 6, 0, 7, 8] 75 22
[1, 9, 3, 4, 5, 2, 6, 0, 7, 8] 75 23
[1, 9, 3, 4, 5, 2, 6, 0, 7, 8] 75 24
[1, 9, 3, 4, 5, 2, 6, 0, 7, 8] 75 25
[1, 9, 3, 4, 5, 2, 6, 0, 7, 8] 75 26
[2, 5, 4, 3

([8, 7, 0, 6, 3, 4, 5, 1, 9, 2], 68, 41)

In [65]:
local_search_with_permutation(graph, random_solution, total_edge_length, 1000, change_func=make_change_swap)

[5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 0
[8, 6, 3, 1, 4, 5, 9, 0, 7, 2] 84 1
[8, 3, 6, 1, 5, 0, 4, 7, 9, 2] 82 2
[8, 3, 7, 0, 1, 6, 4, 5, 9, 2] 81 3
[8, 3, 7, 0, 4, 6, 5, 1, 9, 2] 80 4
[8, 3, 7, 0, 4, 6, 5, 1, 9, 2] 80 5
[8, 3, 7, 0, 4, 6, 5, 1, 9, 2] 80 6
[8, 3, 7, 0, 4, 6, 5, 1, 9, 2] 80 7
[8, 0, 7, 3, 4, 6, 5, 1, 9, 2] 74 8
[8, 0, 7, 3, 6, 1, 4, 5, 9, 2] 73 9
[8, 0, 7, 3, 6, 1, 4, 5, 9, 2] 73 10
[0, 8, 7, 3, 6, 4, 5, 1, 9, 2] 72 11
[0, 8, 7, 3, 6, 4, 5, 1, 9, 2] 72 12
[0, 8, 7, 3, 6, 4, 5, 1, 9, 2] 72 13
[0, 8, 7, 3, 6, 4, 5, 1, 9, 2] 72 14
[0, 8, 7, 3, 6, 4, 5, 1, 9, 2] 72 15
[0, 8, 7, 3, 6, 4, 5, 1, 9, 2] 72 16
[0, 8, 7, 3, 6, 4, 5, 1, 9, 2] 72 17
[0, 8, 7, 3, 6, 4, 5, 1, 9, 2] 72 18
[8, 0, 7, 3, 6, 4, 5, 1, 9, 2] 71 19
[8, 0, 7, 3, 6, 4, 5, 1, 9, 2] 71 20
[8, 0, 7, 3, 6, 4, 5, 1, 9, 2] 71 21
[8, 0, 7, 3, 6, 4, 5, 1, 9, 2] 71 22
[8, 0, 7, 3, 6, 4, 5, 1, 9, 2] 71 23
[8, 0, 7, 3, 6, 4, 5, 1, 9, 2] 71 24
[8, 0, 7, 3, 6, 4, 5, 1, 9, 2] 71 25
[8, 0, 7, 3, 6, 4, 5, 1, 9, 2] 71 26
[8, 0, 7, 3

([8, 0, 7, 6, 3, 4, 5, 1, 9, 2], 68, 58)

In [15]:
def simulated_annealing(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(1, num_iters + 1):
        print(solution, value)
        new_solution = change_func(graph, solution)
        new_value = calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

        elif random.random() < 1 / i:
            print('divs')
            value = new_value
            solution = deepcopy(new_solution)

    return best_solution, best_value, best_i

In [16]:
simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_swap)

[5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86
divs
[5, 6, 3, 7, 4, 8, 9, 0, 2, 1] 90
[5, 6, 3, 7, 4, 8, 9, 0, 2, 1] 90
[9, 6, 3, 7, 4, 8, 5, 0, 2, 1] 88
[9, 6, 3, 7, 4, 8, 1, 0, 2, 5] 87
[9, 6, 3, 7, 4, 8, 1, 0, 2, 5] 87
[9, 6, 3, 7, 4, 8, 1, 0, 2, 5] 87
[9, 2, 3, 7, 4, 8, 1, 0, 6, 5] 79
[9, 2, 3, 7, 4, 8, 1, 0, 6, 5] 79
[2, 9, 3, 7, 4, 8, 1, 0, 6, 5] 77
[2, 9, 3, 7, 4, 8, 1, 0, 6, 5] 77
[2, 9, 3, 7, 4, 8, 1, 5, 6, 0] 75
[2, 9, 3, 7, 4, 8, 1, 5, 6, 0] 75
[2, 9, 3, 7, 4, 8, 1, 5, 6, 0] 75
[2, 9, 3, 7, 4, 8, 1, 5, 6, 0] 75
[2, 9, 3, 7, 4, 8, 1, 5, 6, 0] 75
[2, 9, 3, 7, 4, 8, 1, 5, 6, 0] 75
[2, 9, 3, 7, 4, 8, 1, 5, 6, 0] 75
[2, 9, 3, 7, 4, 8, 1, 5, 6, 0] 75
[2, 9, 3, 7, 4, 8, 1, 5, 6, 0] 75
[2, 9, 3, 7, 4, 8, 1, 5, 6, 0] 75
[2, 9, 3, 7, 4, 8, 1, 5, 6, 0] 75
[2, 9, 3, 7, 4, 8, 1, 5, 6, 0] 75
[2, 9, 3, 7, 4, 8, 1, 5, 6, 0] 75
[2, 9, 3, 7, 4, 8, 1, 5, 6, 0] 75
[2, 9, 3, 7, 4, 8, 1, 5, 6, 0] 75
[2, 9, 3, 7, 4, 8, 1, 5, 6, 0] 75
[2, 9, 3, 7, 4, 8, 1, 5, 6, 0] 75
[2, 9, 3, 7, 4, 8, 1, 5, 6, 0] 75
[2, 9, 3,

([8, 0, 6, 7, 3, 4, 5, 1, 9, 2], 68, 315)

In [17]:
simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse)

[5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86
divs
[5, 6, 0, 9, 8, 4, 1, 3, 2, 7] 95
[5, 6, 0, 9, 8, 1, 4, 3, 2, 7] 94
divs
[5, 6, 4, 1, 8, 9, 0, 3, 2, 7] 96
[5, 6, 4, 1, 8, 9, 0, 3, 2, 7] 96
[0, 9, 8, 1, 4, 6, 5, 3, 2, 7] 94
[0, 9, 8, 1, 4, 6, 5, 3, 2, 7] 94
[0, 5, 6, 4, 1, 8, 9, 3, 2, 7] 87
[0, 8, 1, 4, 6, 5, 9, 3, 2, 7] 85
[0, 8, 1, 4, 6, 5, 9, 3, 2, 7] 85
[2, 3, 9, 5, 6, 4, 1, 8, 0, 7] 79
[2, 3, 9, 5, 6, 4, 1, 0, 8, 7] 78
[2, 3, 9, 5, 6, 4, 1, 0, 8, 7] 78
[2, 3, 9, 5, 6, 4, 1, 0, 8, 7] 78
[2, 3, 9, 5, 6, 4, 1, 0, 8, 7] 78
[2, 3, 9, 5, 6, 4, 1, 0, 8, 7] 78
[2, 3, 9, 5, 6, 4, 1, 0, 8, 7] 78
[2, 5, 9, 3, 6, 4, 1, 0, 8, 7] 76
[2, 5, 9, 3, 6, 4, 1, 0, 8, 7] 76
[2, 5, 9, 3, 6, 4, 1, 0, 8, 7] 76
[2, 5, 9, 3, 6, 4, 1, 0, 8, 7] 76
[2, 5, 9, 3, 6, 4, 1, 0, 8, 7] 76
[2, 5, 9, 3, 6, 4, 1, 0, 8, 7] 76
[2, 5, 9, 3, 4, 6, 1, 0, 8, 7] 75
[2, 5, 9, 3, 4, 6, 1, 0, 8, 7] 75
[2, 5, 9, 3, 4, 6, 1, 0, 8, 7] 75
[2, 5, 9, 3, 4, 6, 1, 0, 8, 7] 75
[2, 5, 9, 3, 4, 6, 1, 0, 8, 7] 75
[2, 5, 9, 3, 4, 6, 1, 0, 8, 7] 75
[2, 

([2, 9, 1, 5, 3, 4, 6, 7, 0, 8], 68, 200)

In [18]:
def shaking_swap(graph, solution, k):
    new_solution = deepcopy(solution)
    selected = random.sample(range(len(solution)), 2*k)
    for i in range(0,len(selected), 2):
        index1, index2 = selected[i], selected[i+1]
        new_solution[index1], new_solution[index2] = new_solution[index2], new_solution[index1]
    return new_solution    
    

In [19]:
def shaking_inverse(graph, solution, k):
    new_solution = deepcopy(solution)
    index1 = random.randint(0, len(solution))
    if(index1 + k > len(solution)):
        index1 = len(solution) - k
    index2 = index1 + k                       
    new_solution[index1:index2+1] = reversed(new_solution[index1:index2+1])  

    return new_solution

In [20]:
def vns(graph, random_solution, value, num_iters, change_func, shaking_func, local_search_func, k_min, k_max, move_prob):
    solution = deepcopy(random_solution)
    best_i = None
    for i in range(num_iters):
        for k in range(k_min, k_max):
            print('vns: ', solution, value, i)
            new_solution = shaking_func(graph, solution, k) #diversification    
            new_value = calculate_total_edge_length(graph, new_solution)
            print('post shaking: ', new_solution, new_value, i)
            new_solution, new_value, _= local_search_func(graph, new_solution, total_edge_length, 10, change_func)
            if new_value < value or (new_value == value and random.random() < move_prob):
                if(new_value < value):
                    best_i = i
                value = new_value
                solution = deepcopy(new_solution)
    return solution, value, best_i

In [21]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_inverse, shaking_func=shaking_swap, local_search_func = local_search, k_min=1, k_max=3, move_prob=0.4)

vns:  [5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 0
post shaking:  [5, 6, 3, 1, 4, 8, 7, 0, 2, 9] 84 0
[5, 6, 3, 1, 4, 8, 7, 0, 2, 9] 86 0
[9, 2, 0, 7, 8, 4, 1, 3, 6, 5] 84 1
[9, 2, 0, 7, 8, 4, 1, 3, 6, 5] 84 2
[9, 2, 0, 7, 8, 4, 1, 3, 6, 5] 84 3
[9, 2, 0, 7, 8, 4, 1, 3, 6, 5] 84 4
[9, 2, 0, 7, 8, 4, 1, 3, 6, 5] 84 5
[9, 2, 0, 7, 8, 4, 1, 3, 6, 5] 84 6
[9, 2, 0, 7, 8, 4, 1, 3, 6, 5] 84 7
[9, 2, 1, 4, 8, 7, 0, 3, 6, 5] 82 8
[9, 2, 1, 4, 8, 7, 0, 3, 6, 5] 82 9
vns:  [9, 2, 1, 4, 8, 7, 0, 3, 6, 5] 82 0
post shaking:  [9, 2, 0, 4, 6, 7, 1, 3, 8, 5] 98 0
[9, 2, 0, 4, 6, 7, 1, 3, 8, 5] 86 0
[9, 2, 0, 4, 6, 7, 1, 3, 8, 5] 86 1
[9, 2, 0, 4, 6, 7, 1, 3, 8, 5] 86 2
[9, 2, 0, 4, 6, 7, 1, 3, 8, 5] 86 3
[9, 2, 0, 4, 6, 7, 1, 3, 8, 5] 86 4
[9, 2, 0, 4, 6, 7, 1, 3, 8, 5] 86 5
[9, 2, 0, 4, 6, 7, 1, 3, 8, 5] 86 6
[9, 2, 0, 4, 6, 7, 1, 3, 8, 5] 86 7
[9, 2, 0, 4, 6, 7, 1, 3, 8, 5] 86 8
[9, 2, 0, 4, 6, 7, 1, 3, 8, 5] 86 9
vns:  [9, 2, 1, 4, 8, 7, 0, 3, 6, 5] 82 1
post shaking:  [2, 9, 1, 4, 8, 7, 0, 3, 6, 5] 80 1
[

([8, 7, 0, 6, 3, 5, 4, 1, 9, 2], 68, 6)

In [22]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_inverse, shaking_func=shaking_inverse, local_search_func = local_search, k_min=3, k_max=6, move_prob=0.4)

vns:  [5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 0
post shaking:  [5, 4, 1, 3, 6, 8, 9, 0, 2, 7] 92 0
[5, 4, 1, 3, 6, 8, 9, 0, 2, 7] 86 0
[5, 4, 1, 3, 6, 8, 9, 0, 2, 7] 86 1
[5, 4, 1, 3, 6, 8, 9, 0, 2, 7] 86 2
[5, 4, 1, 3, 6, 8, 9, 0, 2, 7] 86 3
[5, 4, 1, 3, 6, 8, 9, 0, 2, 7] 86 4
[5, 4, 1, 3, 6, 8, 9, 0, 2, 7] 86 5
[5, 4, 1, 3, 6, 8, 9, 0, 2, 7] 86 6
[5, 4, 1, 3, 6, 8, 9, 0, 2, 7] 86 7
[5, 4, 1, 3, 6, 8, 9, 0, 2, 7] 86 8
[5, 4, 1, 3, 6, 8, 9, 0, 2, 7] 86 9
vns:  [5, 4, 1, 3, 6, 8, 9, 0, 2, 7] 86 0
post shaking:  [5, 4, 1, 3, 6, 8, 7, 2, 0, 9] 94 0
[5, 4, 1, 3, 6, 8, 7, 2, 0, 9] 86 0
[5, 4, 1, 3, 6, 8, 7, 2, 0, 9] 86 1
[5, 4, 1, 3, 6, 8, 7, 2, 0, 9] 86 2
[5, 4, 1, 3, 6, 8, 7, 2, 0, 9] 86 3
[5, 4, 1, 3, 6, 8, 7, 2, 0, 9] 86 4
[5, 4, 1, 3, 6, 8, 7, 2, 0, 9] 86 5
[5, 4, 1, 3, 6, 8, 7, 2, 0, 9] 86 6
[5, 4, 1, 3, 6, 8, 7, 2, 0, 9] 86 7
[5, 4, 1, 3, 6, 8, 7, 2, 0, 9] 86 8
[5, 4, 1, 3, 6, 8, 7, 2, 0, 9] 86 9
vns:  [5, 4, 1, 3, 6, 8, 9, 0, 2, 7] 86 0
post shaking:  [5, 4, 1, 3, 6, 7, 2, 0, 9, 8] 94 0
[

([8, 0, 7, 6, 4, 5, 3, 1, 9, 2], 68, 2)

In [23]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_swap, shaking_func=shaking_swap, local_search_func = local_search, k_min=1, k_max=3, move_prob=0.4)

vns:  [5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 0
post shaking:  [5, 6, 3, 1, 4, 9, 8, 0, 2, 7] 85 0
[5, 6, 3, 1, 4, 9, 8, 0, 2, 7] 86 0
[5, 1, 3, 6, 4, 9, 8, 0, 2, 7] 85 1
[5, 1, 3, 6, 4, 9, 8, 0, 2, 7] 85 2
[5, 1, 3, 6, 4, 9, 8, 0, 7, 2] 83 3
[5, 1, 3, 6, 4, 9, 8, 0, 7, 2] 83 4
[5, 1, 3, 6, 4, 9, 8, 0, 7, 2] 83 5
[5, 1, 3, 6, 4, 9, 8, 0, 7, 2] 83 6
[5, 1, 3, 6, 4, 9, 8, 0, 7, 2] 83 7
[5, 1, 3, 6, 4, 9, 8, 0, 7, 2] 83 8
[5, 1, 3, 6, 4, 9, 8, 0, 7, 2] 83 9
vns:  [5, 1, 3, 6, 4, 9, 8, 0, 7, 2] 83 0
post shaking:  [5, 9, 3, 6, 4, 1, 8, 2, 7, 0] 93 0
[5, 9, 3, 6, 4, 1, 8, 2, 7, 0] 86 0
[5, 9, 3, 6, 4, 1, 8, 2, 7, 0] 86 1
[5, 9, 3, 6, 4, 1, 8, 2, 7, 0] 86 2
[5, 9, 3, 6, 4, 1, 8, 2, 7, 0] 86 3
[5, 9, 3, 6, 4, 1, 8, 2, 7, 0] 86 4
[5, 9, 3, 6, 4, 1, 8, 2, 7, 0] 86 5
[5, 9, 3, 6, 4, 1, 8, 2, 7, 0] 86 6
[5, 9, 3, 6, 4, 1, 8, 2, 7, 0] 86 7
[5, 9, 3, 6, 4, 1, 8, 2, 7, 0] 86 8
[5, 9, 3, 6, 4, 1, 8, 2, 7, 0] 86 9
vns:  [5, 1, 3, 6, 4, 9, 8, 0, 7, 2] 83 1
post shaking:  [5, 1, 3, 8, 4, 9, 6, 0, 7, 2] 92 1
[

([8, 0, 7, 6, 3, 4, 5, 1, 9, 2], 68, 4)

In [24]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_swap, shaking_func=shaking_inverse, local_search_func = local_search, k_min=3, k_max=6, move_prob=0.4)

vns:  [5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 0
post shaking:  [5, 6, 3, 1, 4, 2, 0, 9, 8, 7] 83 0
[5, 6, 3, 1, 4, 2, 0, 9, 8, 7] 86 0
[5, 6, 3, 1, 4, 2, 0, 8, 9, 7] 84 1
[6, 5, 3, 1, 4, 2, 0, 8, 9, 7] 83 2
[6, 5, 3, 1, 4, 2, 0, 8, 9, 7] 83 3
[6, 5, 3, 1, 4, 2, 0, 8, 9, 7] 83 4
[6, 5, 3, 1, 4, 2, 0, 8, 9, 7] 83 5
[6, 5, 3, 1, 4, 2, 0, 8, 9, 7] 83 6
[6, 5, 3, 1, 4, 2, 0, 8, 9, 7] 83 7
[6, 5, 3, 1, 4, 2, 0, 8, 9, 7] 83 8
[6, 5, 3, 1, 4, 2, 0, 8, 9, 7] 83 9
vns:  [6, 5, 3, 1, 4, 2, 0, 8, 9, 7] 83 0
post shaking:  [6, 5, 3, 1, 4, 2, 7, 9, 8, 0] 84 0
[6, 5, 3, 1, 4, 2, 7, 9, 8, 0] 86 0
[6, 5, 3, 1, 4, 2, 7, 9, 8, 0] 86 1
[6, 5, 3, 1, 4, 2, 7, 9, 8, 0] 86 2
[6, 5, 3, 1, 4, 2, 7, 9, 8, 0] 86 3
[6, 5, 3, 1, 4, 2, 7, 9, 8, 0] 86 4
[6, 5, 3, 1, 4, 2, 7, 9, 8, 0] 86 5
[6, 5, 3, 1, 4, 8, 7, 9, 2, 0] 85 6
[6, 5, 3, 1, 4, 8, 7, 9, 2, 0] 85 7
[6, 5, 3, 1, 4, 8, 7, 9, 2, 0] 85 8
[6, 5, 3, 1, 4, 9, 7, 8, 2, 0] 83 9
vns:  [6, 5, 3, 1, 4, 2, 0, 8, 9, 7] 83 0
post shaking:  [6, 5, 3, 1, 4, 7, 9, 8, 0, 2] 83 0
[

([2, 9, 1, 5, 3, 4, 6, 0, 7, 8], 68, 8)

In [68]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_swap, shaking_func=shaking_swap, local_search_func = local_search_with_permutation, k_min=1, k_max=3, move_prob=0.4)

vns:  [5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 0
post shaking:  [5, 6, 3, 8, 4, 1, 9, 0, 2, 7] 88 0
[5, 6, 3, 8, 4, 1, 9, 0, 2, 7] 86 0
[5, 6, 3, 8, 4, 1, 9, 0, 2, 7] 86 1
[5, 6, 3, 8, 4, 1, 9, 0, 2, 7] 86 2
[6, 5, 3, 8, 4, 1, 9, 0, 7, 2] 85 3
[6, 5, 3, 8, 4, 1, 9, 0, 7, 2] 85 4
[6, 5, 3, 8, 4, 1, 9, 0, 7, 2] 85 5
[6, 5, 3, 8, 4, 1, 9, 0, 7, 2] 85 6
[6, 5, 3, 1, 4, 8, 9, 0, 7, 2] 83 7
[6, 5, 3, 1, 4, 8, 9, 0, 7, 2] 83 8
[6, 5, 3, 1, 4, 8, 9, 0, 7, 2] 83 9
vns:  [6, 5, 3, 1, 4, 8, 9, 0, 7, 2] 83 0
post shaking:  [6, 1, 3, 5, 4, 0, 9, 8, 7, 2] 83 0
[6, 1, 3, 5, 4, 0, 9, 8, 7, 2] 86 0
[6, 1, 3, 4, 5, 0, 9, 8, 7, 2] 85 1
[6, 1, 3, 4, 5, 0, 9, 8, 7, 2] 85 2
[6, 1, 3, 4, 5, 0, 9, 8, 7, 2] 85 3
[6, 1, 3, 4, 5, 0, 9, 8, 7, 2] 85 4
[5, 1, 3, 4, 6, 0, 9, 8, 7, 2] 83 5
[5, 1, 3, 4, 6, 0, 9, 8, 7, 2] 83 6
[5, 1, 3, 4, 6, 0, 9, 8, 7, 2] 83 7
[5, 1, 3, 4, 6, 9, 2, 0, 7, 8] 81 8
[5, 1, 3, 4, 6, 9, 2, 0, 7, 8] 81 9
vns:  [5, 1, 3, 4, 6, 9, 2, 0, 7, 8] 81 1
post shaking:  [5, 1, 3, 4, 6, 9, 8, 0, 7, 2] 84 1
[

([8, 0, 7, 6, 4, 5, 3, 1, 9, 2], 68, 5)

In [70]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_swap, shaking_func=shaking_inverse, local_search_func = local_search_with_permutation, k_min=3, k_max=6, move_prob=0.4)

vns:  [5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 0
post shaking:  [5, 6, 3, 1, 4, 8, 9, 7, 2, 0] 86 0
[5, 6, 3, 1, 4, 8, 9, 7, 2, 0] 86 0
[5, 6, 3, 1, 4, 8, 9, 7, 2, 0] 86 1
[5, 6, 3, 1, 4, 8, 7, 0, 2, 9] 84 2
[5, 6, 3, 1, 4, 8, 7, 0, 2, 9] 84 3
[5, 6, 3, 1, 4, 0, 7, 8, 2, 9] 82 4
[5, 6, 3, 1, 4, 0, 7, 8, 2, 9] 82 5
[5, 6, 3, 1, 4, 0, 7, 8, 2, 9] 82 6
[6, 5, 3, 1, 4, 0, 7, 8, 2, 9] 81 7
[6, 5, 3, 1, 4, 0, 7, 8, 9, 2] 79 8
[6, 5, 3, 1, 4, 0, 7, 8, 9, 2] 79 9
vns:  [6, 5, 3, 1, 4, 0, 7, 8, 9, 2] 79 0
post shaking:  [6, 5, 7, 0, 4, 1, 3, 8, 9, 2] 89 0
[6, 5, 7, 0, 4, 1, 3, 8, 9, 2] 86 0
[6, 5, 7, 0, 4, 1, 3, 8, 9, 2] 86 1
[6, 5, 0, 7, 4, 1, 3, 8, 9, 2] 85 2
[6, 5, 0, 7, 4, 1, 3, 8, 9, 2] 85 3
[6, 5, 0, 7, 4, 1, 3, 8, 9, 2] 85 4
[6, 5, 0, 7, 4, 1, 3, 8, 9, 2] 85 5
[6, 5, 0, 7, 4, 1, 3, 8, 9, 2] 85 6
[6, 5, 0, 7, 4, 1, 3, 8, 9, 2] 85 7
[6, 5, 0, 7, 4, 1, 3, 8, 9, 2] 85 8
[6, 5, 0, 7, 4, 1, 3, 8, 9, 2] 85 9
vns:  [6, 5, 3, 1, 4, 0, 7, 8, 9, 2] 79 0
post shaking:  [6, 5, 3, 1, 4, 2, 9, 8, 7, 0] 81 0
[

([8, 6, 0, 7, 4, 3, 5, 1, 9, 2], 68, 11)

In [71]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_inverse, shaking_func=shaking_swap, local_search_func = local_search_with_permutation, k_min=1, k_max=3, move_prob=0.4)

vns:  [5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 0
post shaking:  [5, 6, 3, 1, 2, 8, 9, 0, 4, 7] 96 0
[5, 6, 3, 1, 2, 8, 9, 0, 4, 7] 86 0
[5, 6, 3, 1, 2, 8, 9, 0, 4, 7] 86 1
[5, 6, 3, 1, 2, 8, 9, 0, 4, 7] 86 2
[5, 6, 3, 1, 2, 8, 9, 0, 4, 7] 86 3
[5, 6, 3, 1, 2, 8, 9, 0, 4, 7] 86 4
[5, 6, 3, 1, 2, 8, 9, 0, 4, 7] 86 5
[5, 6, 3, 1, 2, 8, 9, 0, 4, 7] 86 6
[5, 6, 3, 1, 2, 8, 9, 0, 4, 7] 86 7
[5, 6, 3, 1, 2, 8, 9, 0, 4, 7] 86 8
[5, 6, 3, 1, 2, 8, 9, 0, 4, 7] 86 9
vns:  [5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 0
post shaking:  [5, 6, 8, 9, 4, 3, 1, 0, 2, 7] 96 0
[5, 6, 8, 9, 4, 3, 1, 0, 2, 7] 86 0
[5, 6, 8, 9, 4, 3, 1, 0, 2, 7] 86 1
[5, 6, 8, 9, 4, 3, 1, 0, 2, 7] 86 2
[5, 6, 8, 9, 4, 3, 1, 0, 2, 7] 86 3
[5, 6, 8, 9, 4, 3, 1, 0, 2, 7] 86 4
[5, 6, 8, 9, 4, 3, 1, 0, 2, 7] 86 5
[5, 6, 8, 9, 4, 3, 1, 0, 2, 7] 86 6
[5, 6, 8, 9, 4, 3, 1, 0, 2, 7] 86 7
[5, 6, 8, 9, 4, 3, 1, 0, 2, 7] 86 8
[5, 6, 0, 1, 3, 4, 9, 8, 2, 7] 81 9
vns:  [5, 6, 0, 1, 3, 4, 9, 8, 2, 7] 81 1
post shaking:  [3, 6, 0, 1, 5, 4, 9, 8, 2, 7] 85 1
[

([8, 6, 0, 7, 4, 3, 5, 1, 9, 2], 68, 31)

In [72]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_inverse, shaking_func=shaking_inverse, local_search_func = local_search_with_permutation, k_min=3, k_max=6, move_prob=0.4)

vns:  [5, 6, 3, 1, 4, 8, 9, 0, 2, 7] 86 0
post shaking:  [5, 4, 1, 3, 6, 8, 9, 0, 2, 7] 92 0
[5, 4, 1, 3, 6, 8, 9, 0, 2, 7] 86 0
[5, 4, 1, 3, 6, 8, 9, 0, 2, 7] 86 1
[5, 4, 1, 3, 6, 8, 9, 0, 2, 7] 86 2
[5, 4, 1, 3, 6, 8, 9, 0, 2, 7] 86 3
[5, 4, 1, 3, 6, 8, 9, 0, 2, 7] 86 4
[5, 4, 1, 3, 6, 8, 9, 0, 2, 7] 86 5
[5, 4, 1, 3, 6, 8, 9, 0, 2, 7] 86 6
[5, 4, 1, 3, 6, 8, 9, 0, 2, 7] 86 7
[5, 4, 1, 3, 6, 8, 9, 0, 2, 7] 86 8
[5, 4, 1, 3, 6, 8, 9, 0, 2, 7] 86 9
vns:  [5, 4, 1, 3, 6, 8, 9, 0, 2, 7] 86 0
post shaking:  [5, 4, 1, 3, 6, 7, 2, 0, 9, 8] 94 0
[5, 4, 1, 3, 6, 7, 2, 0, 9, 8] 86 0
[5, 4, 1, 3, 6, 7, 2, 0, 9, 8] 86 1
[5, 4, 1, 3, 6, 7, 2, 0, 9, 8] 86 2
[5, 4, 1, 3, 6, 7, 2, 0, 9, 8] 86 3
[5, 4, 1, 3, 6, 7, 2, 0, 9, 8] 86 4
[5, 4, 1, 3, 6, 7, 2, 0, 9, 8] 86 5
[5, 4, 1, 3, 6, 7, 2, 0, 9, 8] 86 6
[5, 4, 1, 3, 6, 7, 2, 0, 9, 8] 86 7
[5, 4, 1, 3, 6, 7, 2, 0, 9, 8] 86 8
[5, 4, 1, 3, 6, 7, 2, 0, 9, 8] 86 9
vns:  [5, 4, 1, 3, 6, 7, 2, 0, 9, 8] 86 0
post shaking:  [5, 4, 1, 3, 8, 9, 0, 2, 7, 6] 101 0


([8, 0, 7, 6, 4, 5, 3, 1, 9, 2], 68, 7)

In [74]:
#genetic algorithms

In [76]:
class Individual:
    def __init__(self, num_nodes, graph):
        self.num_nodes = num_nodes
        self.graph = graph
        self.code = self.generate_solution(self.graph)
        self.fitness = self.calc_fitness(self.code)
    
    def generate_solution(self, num_nodes):
        nodes = list(range(num_nodes))
        random.shuffle(nodes)
        return nodes

    def calc_fitness(self, solution):
        total_length = 0
        for i in range(len(graph)):
            for j in range(i + 1, len(graph)):
                if graph[i][j] == 1:
                    position_i = solution.index(i)
                    position_j = solution.index(j)
                    total_length += abs(position_i - position_j)
        return total_length

In [77]:
def tournament_selection(population, tour_size):
    tournament = random.sample(population, tour_size)
    selected = max(tournament, key=lambda x: x.fitness)
    return selected

In [79]:
#beacause we are dealing with permutations, we have to be careful here and used sepcialized crossover algorithms
#ordered crossover method
#PROVERI OVO
def ordered_crossover(parent1, parent2):
    n = len(parent1)

    # Choose two random crossover points
    cx_point1, cx_point2 = sorted(random.sample(range(n), 2))

    # Copy the segment between the crossover points from parent1 to child1
    child1 = parent1[cx_point1:cx_point2 + 1]

    # Fill the remaining positions in child1 with elements from parent2
    remaining_positions = set(range(n)) - set(child1)
    for pos in range(n):
        if len(child1) == n:
            break
        if pos < cx_point1 or pos > cx_point2:
            if parent2[pos] not in child1:
                child1.append(parent2[pos])

    # Create child2 by swapping the roles of parent1 and parent2
    child2 = ordered_crossover(parent2, parent1)

    return child1, child2
